# Pydantic Variants Exercise

This notebook demonstrates the usage of the pydantic_variants package, focusing on:
1. Basic variant creation using the @variants decorator
2. Nested model variants with field transformations
3. **3-level nested variant switches** - the main test case
4. Field filtering, optional conversion, and attribute modification

## Test Goals
- Test the `@variants` class decorator
- Create complex nested model hierarchies
- Demonstrate variant switching at multiple levels
- Validate field transformations work correctly

In [3]:
# Import Required Libraries and Package
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
import sys

sys.path.insert(0, r"c:\dev")
from pydantic_variants import *  # noqa: F403
from pydantic_variants.transformers import *  # noqa: F403

# Add parent directory to path to import our package


print("✅ All imports successful!")


ModuleNotFoundError: No module named 'decorators'

In [21]:
# Build Pipelines Outside Decorators
print("🔧 Building variant pipelines...\n")

# Level 3 (ContactInfo) Pipelines
contact_input_pipeline = basic_variant_pipeline('Input', 
    FilterFields(exclude=['id', 'verified_at']),
    MakeOptional(all=True)
)

contact_output_pipeline = basic_variant_pipeline('Output',
    FilterFields(exclude=['internal_notes'])
)

contact_update_pipeline = basic_variant_pipeline('Update',
    FilterFields(exclude=['id', 'created_at']),
    MakeOptional(all=True)
)

print("✅ Level 3 (ContactInfo) pipelines created")

# Level 2 (Address) Pipelines
address_input_pipeline = basic_variant_pipeline('Input',
    FilterFields(exclude=['id', 'created_at']),
    SwitchNested('contact', 'Input'),  # Switch nested ContactInfo to Input variant
    MakeOptional(fields=['country'])
)

address_output_pipeline = basic_variant_pipeline('Output', 
    FilterFields(exclude=['internal_id']),
    SwitchNested('contact', 'Output')  # Switch nested ContactInfo to Output variant
)

address_update_pipeline = basic_variant_pipeline('Update',
    FilterFields(exclude=['id', 'created_at']),
    SwitchNested('contact', 'Update'),  # Switch nested ContactInfo to Update variant
    MakeOptional(all=True)
)

print("✅ Level 2 (Address) pipelines created")

# Level 1 (User) Pipelines  
user_input_pipeline = basic_variant_pipeline('Input',
    FilterFields(exclude=['id', 'created_at', 'last_login']),
    SwitchNested('address', 'Input'),  # Level 1 → Level 2 → Level 3 variant switching
    MakeOptional(fields=['age', 'bio'])
)

user_output_pipeline = basic_variant_pipeline('Output',
    FilterFields(exclude=['password_hash', 'internal_notes']),
    SwitchNested('address', 'Output'),  # Cascade Output variants down all levels
    SetFields(display_name=str)  # Add computed field for output
)

user_update_pipeline = basic_variant_pipeline('Update', 
    FilterFields(exclude=['id', 'created_at']),
    SwitchNested('address', 'Update'),  # Cascade Update variants down all levels
    MakeOptional(all=True)
)

user_admin_pipeline = basic_variant_pipeline('Admin',
    # Admin variant shows everything, no filtering
    SwitchNested('address', 'Output'),  # Use clean output for nested data
    SetFields(admin_notes=str)  # Add admin-specific field
)

print("✅ Level 1 (User) pipelines created")
print("\n🎯 All pipelines ready for 3-level nested variant switching!")

🔧 Building variant pipelines...



ImportError: attempted relative import with no known parent package

In [ ]:
# Define Level 3 (Deepest) Model - Contact Info
@variants(
    contact_input_pipeline,
    contact_output_pipeline,
    contact_update_pipeline
)
class ContactInfo(BaseModel):
    """Level 3: Contact information with variants"""
    id: int
    email: str
    phone: Optional[str] = None
    verified_at: Optional[datetime] = None
    internal_notes: str = ""
    created_at: datetime = Field(default_factory=datetime.now)

print("✅ Level 3 model (ContactInfo) created with variants:")
print(f"   - Available variants: {list(getattr(ContactInfo, '_variants', {}).keys())}")
print(f"   - Input variant: {ContactInfo._Input}")
print(f"   - Output variant: {ContactInfo._Output}")
print(f"   - Update variant: {ContactInfo._Update}")

In [ ]:
# Define Level 2 Model - Address with nested ContactInfo
@variants(
    address_input_pipeline,
    address_output_pipeline,
    address_update_pipeline
)
class Address(BaseModel):
    """Level 2: Address with nested contact info and variants"""
    id: int
    street: str
    city: str
    country: str = "USA"
    postal_code: str
    contact: ContactInfo
    internal_id: str = "ADDR_"
    created_at: datetime = Field(default_factory=datetime.now)

print("✅ Level 2 model (Address) created with variants:")
print(f"   - Available variants: {list(getattr(Address, '_variants', {}).keys())}")
print("   - Nested model switching: ContactInfo variants used in Address variants")

In [ ]:
# Define Level 1 (Top Level) Model - User with 3-level nesting
@variants(
    user_input_pipeline,
    user_output_pipeline,
    user_update_pipeline,
    user_admin_pipeline
)
class User(BaseModel):
    """Level 1: Top-level User model with 3-level nested variant switching"""
    id: int
    username: str
    email: str
    age: Optional[int] = None
    bio: Optional[str] = None
    address: Address  # Contains ContactInfo → 3 levels total
    password_hash: str = "hashed_password"
    internal_notes: str = ""
    created_at: datetime = Field(default_factory=datetime.now)
    last_login: Optional[datetime] = None

print("🎯 Level 1 model (User) created with 3-LEVEL NESTED VARIANT SWITCHING!")
print(f"   - Available variants: {list(getattr(User, '_variants', {}).keys())}")
print("   - Nesting: User → Address → ContactInfo")
print("   - Each level can switch to different variants independently")

In [ ]:
# Create Sample Data for Testing
print("🔧 Creating sample data for 3-level variant testing...\n")

# Create a full User instance with all nested data
contact_data = ContactInfo(
    id=1,
    email="john@example.com",
    phone="+1-555-0123",
    verified_at=datetime.now(),
    internal_notes="VIP customer",
    created_at=datetime.now()
)

address_data = Address(
    id=1,
    street="123 Main St",
    city="New York",
    country="USA",
    postal_code="10001",
    contact=contact_data,
    internal_id="ADDR_NYC001",
    created_at=datetime.now()
)

user_data = User(
    id=1,
    username="johndoe",
    email="john@example.com",
    age=30,
    bio="Software developer",
    address=address_data,
    password_hash="super_secure_hash",
    internal_notes="Premium user since 2023",
    created_at=datetime.now(),
    last_login=datetime.now()
)

print("✅ Sample data created successfully!")
print(f"   - User: {user_data.username}")
print(f"   - Address: {user_data.address.city}, {user_data.address.country}")
print(f"   - Contact: {user_data.address.contact.email}")

In [ ]:
# Test Input Variants (3-Level Cascade)
print("🔍 Testing INPUT variants across all 3 levels...\n")

# Test Level 3 Input (ContactInfo.Input)
contact_input_data = {
    "email": "jane@example.com",
    "phone": "+1-555-0456"
    # Note: id, verified_at excluded, internal_notes excluded, all optional
}

contact_input = ContactInfo._Input(**contact_input_data)
print("✅ Level 3 Input (ContactInfo._Input):")
print(f"   Fields: {list(contact_input.model_fields.keys())}")
print(f"   Data: {contact_input.model_dump()}\n")

# Test Level 2 Input (Address.Input with nested ContactInfo.Input)
address_input_data = {
    "street": "456 Oak Ave",
    "city": "Boston", 
    "postal_code": "02101",
    "contact": contact_input_data  # Will be converted to ContactInfo.Input
    # Note: id, created_at excluded, country optional
}

address_input = Address._Input(**address_input_data)
print("✅ Level 2 Input (Address._Input with nested ContactInfo._Input):")
print(f"   Fields: {list(address_input.model_fields.keys())}")
print(f"   Nested contact type: {type(address_input.contact)}")
print(f"   Data: {address_input.model_dump()}\n")

# Test Level 1 Input (User.Input with full 3-level cascade)  
user_input_data = {
    "username": "janedoe",
    "email": "jane@example.com",
    "address": address_input_data  # Will cascade to Address.Input → ContactInfo.Input
    # Note: id, created_at, last_login excluded, age/bio optional
}

user_input = User._Input(**user_input_data)
print("🎯 Level 1 Input (User._Input with 3-LEVEL CASCADE):")
print(f"   Fields: {list(user_input.model_fields.keys())}")
print(f"   Nested address type: {type(user_input.address)}")
print(f"   Nested contact type: {type(user_input.address.contact)}")
print("   Full data structure:")
print(f"   {user_input.model_dump(indent=2)}")

In [ ]:
# Test Output Variants (3-Level Cascade)
print("🔍 Testing OUTPUT variants across all 3 levels...\n")

# Convert our original data to Output variants
user_output = User._Output(**user_data.model_dump())
print("🎯 Level 1 Output (User._Output with 3-LEVEL CASCADE):")
print(f"   Fields: {list(user_output.model_fields.keys())}")
print("   Excluded: password_hash, internal_notes")
print(f"   Nested address type: {type(user_output.address)}")
print(f"   Nested contact type: {type(user_output.address.contact)}")

# Check what fields are excluded at each level
print("\n📊 Field Filtering Analysis:")
print("   Level 1 (User): Excluded password_hash, internal_notes")
print("   Level 2 (Address): Excluded internal_id") 
print("   Level 3 (ContactInfo): Excluded internal_notes")

# Show the clean output data
print("\n✅ Clean Output Data Structure:")
output_data = user_output.model_dump()
print(f"   User fields: {list(output_data.keys())}")
print(f"   Address fields: {list(output_data['address'].keys())}")
print(f"   Contact fields: {list(output_data['address']['contact'].keys())}")
print("\n   Full clean data:")
import json
print(json.dumps(output_data, indent=2, default=str))

In [ ]:
# Test Update Variants and Mixed Usage
print("🔍 Testing UPDATE variants and mixed variant usage...\n")

# Test Update cascade
update_data = {
    "username": "updated_user",
    "bio": "Updated bio",
    "address": {
        "street": "789 Updated St",
        "contact": {
            "email": "updated@example.com"
        }
    }
}

user_update = User._Update(**update_data)
print("🎯 Level 1 Update (User._Update with 3-LEVEL CASCADE):")
print(f"   All fields optional: {all(field.default is not ... or field.default_factory for field in user_update.model_fields.values())}")
print("   Nested types: Address._Update → ContactInfo._Update")
print(f"   Data: {user_update.model_dump(exclude_none=True)}\n")

# Test Admin variant (Level 1 only)
user_admin = User._Admin(**user_data.model_dump())
print("🔐 Admin Variant (shows everything + admin fields):")
print(f"   Fields: {list(user_admin.model_fields.keys())}")
print(f"   Has admin_notes field: {'admin_notes' in user_admin.model_fields}")
print(f"   Nested address type: {type(user_admin.address)}")

# Demonstrate variant independence
print("\n🔄 Variant Independence Test:")
print(f"   User._Input type: {User._Input}")
print(f"   User._Output type: {User._Output}")  
print(f"   User._Update type: {User._Update}")
print(f"   User._Admin type: {User._Admin}")
print(f"   All different classes: {len(set([User._Input, User._Output, User._Update, User._Admin])) == 4}")

In [ ]:
# Comprehensive Assertions and Validation
print("🧪 Running comprehensive assertions to validate 3-level variant behavior...\n")

def test_field_exclusions():
    """Test that field exclusions work correctly at each level"""
    print("Testing field exclusions...")
    
    # Level 1 Input should exclude id, created_at, last_login
    user_input_fields = set(User._Input.model_fields.keys())
    assert 'id' not in user_input_fields, "User._Input should exclude 'id'"
    assert 'created_at' not in user_input_fields, "User._Input should exclude 'created_at'"
    assert 'last_login' not in user_input_fields, "User._Input should exclude 'last_login'"
    
    # Level 1 Output should exclude password_hash, internal_notes
    user_output_fields = set(User._Output.model_fields.keys())
    assert 'password_hash' not in user_output_fields, "User._Output should exclude 'password_hash'"
    assert 'internal_notes' not in user_output_fields, "User._Output should exclude 'internal_notes'"
    
    print("   ✅ Field exclusions working correctly")

def test_nested_variant_switching():
    """Test that nested models use correct variants"""
    print("Testing nested variant switching...")
    
    # Create User._Input and check nested types
    user_input = User._Input(
        username="test",
        email="test@example.com", 
        address={
            "street": "Test St",
            "city": "Test City",
            "postal_code": "12345",
            "contact": {"email": "contact@test.com"}
        }
    )
    
    # Check 3-level nesting
    assert type(user_input.address).__name__.endswith('Input'), f"Address should be Input variant, got {type(user_input.address)}"
    assert type(user_input.address.contact).__name__.endswith('Input'), f"Contact should be Input variant, got {type(user_input.address.contact)}"
    
    print("   ✅ 3-level nested variant switching working correctly")

def test_optional_fields():
    """Test that MakeOptional transformer works"""
    print("Testing optional field behavior...")
    
    # User._Update should have all fields optional
    update_fields = User._Update.model_fields
    optional_count = sum(1 for field in update_fields.values() 
                        if field.default is not ... or field.default_factory is not None)
    total_count = len(update_fields)
    
    print(f"   Update variant: {optional_count}/{total_count} fields are optional")
    assert optional_count == total_count, "All fields in Update variant should be optional"
    
    print("   ✅ Optional field behavior working correctly")

def test_variant_independence():
    """Test that variants are independent classes"""
    print("Testing variant independence...")
    
    variants = [User._Input, User._Output, User._Update, User._Admin]
    variant_names = [v.__name__ for v in variants]
    
    # All should be different classes
    assert len(set(variants)) == len(variants), "All variants should be different classes"
    
    # All should have different names
    assert len(set(variant_names)) == len(variant_names), "All variants should have different names"
    
    print(f"   Variant classes: {variant_names}")
    print("   ✅ Variant independence working correctly")

# Run all tests
try:
    test_field_exclusions()
    test_nested_variant_switching() 
    test_optional_fields()
    test_variant_independence()
    
    print("\n🎉 ALL TESTS PASSED!")
    print("   ✅ 3-level nested variant switching is working perfectly")
    print("   ✅ Field transformations are applied correctly")
    print("   ✅ @variants decorator successfully created all variants")
    
except AssertionError as e:
    print(f"\n❌ TEST FAILED: {e}")
except Exception as e:
    print(f"\n💥 UNEXPECTED ERROR: {e}")

In [ ]:
# Summary and Advanced Usage Examples
print("📋 EXERCISE SUMMARY - 3-Level Nested Variant Switching\n")

print("🏗️  Architecture Demonstrated:")
print("   Level 1: User (top-level)")
print("   Level 2: Address (nested in User)")  
print("   Level 3: ContactInfo (nested in Address)")
print("   → Each level has its own variants that cascade down\n")

print("🎯 Variants Created:")
print("   • Input: For data creation (excludes IDs, timestamps)")
print("   • Output: For API responses (excludes sensitive/internal fields)")
print("   • Update: For data modification (all fields optional)")
print("   • Admin: For administrative access (shows everything + admin fields)\n")

print("🔄 Transformations Applied:")
print("   • FilterFields: Remove unwanted fields at each level")
print("   • MakeOptional: Make fields optional for flexible updates")
print("   • SwitchNested: Cascade variant selection to nested models")
print("   • SetFields: Add computed/additional fields\n")

print("✨ Key Benefits Demonstrated:")
print("   1. Declarative variant creation with @variants decorator")
print("   2. Automatic nested variant switching")
print("   3. Type safety maintained across all variants")
print("   4. Clean separation of concerns (Input/Output/Update/Admin)")
print("   5. Reusable pipeline components\n")

# Real-world usage example
print("🌟 Real-world Usage Pattern:")
print("""
# API Endpoint Example:
@app.post("/users/", response_model=User._Output)
async def create_user(user_data: User._Input):
    # user_data has no id, timestamps, sensitive fields
    # but includes all necessary fields for creation
    
    # Create user with full model
    user = User(**user_data.model_dump(), 
                id=generate_id(),
                created_at=datetime.now())
    
    # Return clean output (no sensitive data)
    return User._Output(**user.model_dump())

@app.patch("/users/{user_id}", response_model=User._Output) 
async def update_user(user_id: int, updates: User._Update):
    # updates has all fields optional
    # nested address/contact updates handled automatically
    
    user = get_user(user_id)
    updated_data = user.model_dump()
    updated_data.update(updates.model_dump(exclude_none=True))
    
    user = User(**updated_data)
    return User._Output(**user.model_dump())
""")